In [1]:
import matplotlib as plt
import numpy as np
import pandas as pd

In [2]:
##
train_transaction = pd.read_csv("../data/assembled_data/train_transaction.csv",)
train_transaction.head()

,TransactionID,isFraud,TransactionDT,TransactionAmt,ProductCD,card1,card2,card3,card4,card5,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
0,2987000,0,86400,68.5,W,13926,NaN,150.0,discover,142.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2987001,0,86401,29.0,W,2755,404.0,150.0,mastercard,102.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2987002,0,86469,59.0,W,4663,490.0,150.0,visa,166.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2987003,0,86499,50.0,W,18132,567.0,150.0,mastercard,117.0,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2987004,0,86506,50.0,H,4497,514.0,150.0,mastercard,102.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [3]:
## checking the number of na columns
train_transaction.describe()

,TransactionID,isFraud,TransactionDT,TransactionAmt,card1,card2,card3,card5,addr1,addr2,...,V330,V331,V332,V333,V334,V335,V336,V337,V338,V339
count,5.905400e+05,590540.000000,5.905400e+05,590540.000000,590540.000000,581607.000000,588975.000000,586281.000000,524834.000000,524834.000000,...,82351.000000,82351.000000,82351.000000,82351.000000,82351.000000,82351.00000,82351.000000,82351.000000,82351.000000,82351.000000
mean,3.282270e+06,0.034990,7.372311e+06,135.027176,9898.734658,362.555488,153.194925,199.278897,290.733794,86.800630,...,0.775874,721.741883,1375.783644,1014.622782,9.807015,59.16455,28.530903,55.352422,151.160542,100.700882
std,1.704744e+05,0.183755,4.617224e+06,239.162522,4901.170153,157.793246,11.336444,41.244453,101.741072,2.690623,...,4.727971,6217.223583,11169.275702,7955.735482,243.861391,387.62948,274.576920,668.486833,1095.034387,814.946722
min,2.987000e+06,0.000000,8.640000e+04,0.251000,1000.000000,100.000000,100.000000,100.000000,100.000000,10.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
25%,3.134635e+06,0.000000,3.027058e+06,43.321000,6019.000000,214.000000,150.000000,166.000000,204.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
50%,3.282270e+06,0.000000,7.306528e+06,68.769000,9678.000000,361.000000,150.000000,226.000000,299.000000,87.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
75%,3.429904e+06,0.000000,1.124662e+07,125.000000,14184.000000,512.000000,150.000000,226.000000,330.000000,87.000000,...,0.000000,0.000000,25.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000
max,3.577539e+06,1.000000,1.581113e+07,31937.391000,18396.000000,600.000000,231.000000,237.000000,540.000000,102.000000,...,55.000000,160000.000000,160000.000000,160000.000000,55125.000000,55125.00000,55125.000000,104060.000000,104060.000000,104060.000000


In [4]:
train_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [4]:
import math
import gc
import pickle
import numpy as np
from sklearn.base import BaseEstimator, TransformerMixin
import os


class DataMemoryReducer(BaseEstimator,TransformerMixin):
    '''
    This module attempts to reduce 64 bit datatypes to 32 bit during EDA with pandas so it will fit memory. It will also save the schema to given location
    '''
    def __init__(self,schema_location=None):
        self.float_columns_to_reduce = []
        self.data_schema = {}
        self.schema_location = schema_location
        self.int_columns_to_reduce  = []
        self.__float_min = -3.4028235E+38
        self.__float_max =  3.4028235e+38
        self.__int_min = -2147483648
        self.__int_max = 2147483647
        
            
    def fit(self, pandas_df):
        self.data_schema = pandas_df.dtypes.to_dict()
        float_df = pandas_df.select_dtypes(include='float64')
        float_columns = float_df.columns
        if len(float_columns) == 0:
            print("None of the float columns can be changed")
            
        else: 
            #if all the columns are within range, change all
            if  (np.nanmax(float_df.values) <= self.__float_max) and (np.nanmin(float_df.values) >= self.__float_min):
                self.float_columns_to_reduce = float_columns
                for converted_column in float_columns:
                    self.data_schema[converted_column] = np.dtype("float32")

            else:
                self.float_columns_to_reduce = [] #clear any residuals
                for float_column in float_columns:
                    if ((np.nanmax(float_df[float_column].values) <= self.__float_max) and 
                       (np.nanmin(float_df[float_column].values) >= self.__float_min)):
                        self.float_columns_to_reduce.append(float_column)
                        self.data_schema[float_column] = np.dtype("float32")
                     
        del float_df
        gc.collect()
        
        
        int_df = pandas_df.select_dtypes(include='int64')
        int_columns =int_df.columns
        if len(int_columns) == 0:
            print("None of the int columns can be changed")
        else:    
            #if all the columns are within range, change all
            if  (np.nanmax(int_df.values) <= self.__int_max) and (np.nanmin(int_df.values) >= self.__int_min):
                self.int_columns_to_reduce = int_df.columns
                for converted_column in int_columns:
                    self.data_schema[converted_column] = np.dtype("int32")
                
            #check each column one by one
            else:
                self.int_columns_to_reduce = []
                for int_column in int_columns:
                    if ((np.nanmax(int_df[int_column].values) <= self.__int_max) and 
                       (np.nanmin(int_df[int_column].values) >= self.__int_min)):
                        self.int_columns_to_reduce.append(int_column)
                        self.data_schema[int_column] = np.dtype("int32")
            
            
        del int_df
        gc.collect()
        
        if self.schema_location is not None:
            with open(os.path.join(self.schema_location,"raw_data_schema.pkl"),"wb") as handler:
                pickle.dump(self.data_schema,handler)
    
        return self

    def transform(self, pandas_df):
        temp_df = pandas_df.copy()
        temp_df[self.float_columns_to_reduce]= temp_df[self.float_columns_to_reduce].astype('float32')
        gc.collect()
        temp_df[self.int_columns_to_reduce] = temp_df[self.int_columns_to_reduce].astype('int32')
        gc.collect()
        return temp_df
    
#     def fit_transform(self, pandas_df):
#         self.fit(pandas_df)
#         return self.transform(pandas_df)
        
        

In [5]:
data_memory_reducer = DataMemoryReducer("")

reduced_df = data_memory_reducer.fit_transform(train_transaction)
gc.collect()

0

In [6]:
with open("schema.pkl", "rb") as handler:
    schema = pickle.load(handler)

In [8]:
test_reading = pd.read_csv("../data/assembled_data/train_transaction.csv",dtype=schema)
test_reading.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float32(376), int32(4), object(14)
memory usage: 919.1+ MB


In [11]:
reduced_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float32(376), int32(4), object(14)
memory usage: 919.1+ MB


In [12]:
train_transaction.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 590540 entries, 0 to 590539
Columns: 394 entries, TransactionID to V339
dtypes: float64(376), int64(4), object(14)
memory usage: 1.7+ GB


In [13]:
data_memory_reducer.float_columns_to_reduce

Index(['TransactionAmt', 'card2', 'card3', 'card5', 'addr1', 'addr2', 'dist1',
       'dist2', 'C1', 'C2',
       ...
       'V330', 'V331', 'V332', 'V333', 'V334', 'V335', 'V336', 'V337', 'V338',
       'V339'],
      dtype='object', length=376)

In [14]:
data_memory_reducer.int_columns_to_reduce

Index(['TransactionID', 'isFraud', 'TransactionDT', 'card1'], dtype='object')

In [14]:
from sklearn.impute import SimpleImputer

simple_imputer = SimpleImputer()
test_2 = simple_imputer.fit_transform(reduced_df.select_dtypes("float32").values)


AttributeError: 'numpy.ndarray' object has no attribute 'head'

In [15]:
test_2

array([[ 68.5     , 362.5555  , 150.      , ...,  55.352417, 151.16054 ,
        100.700874],
       [ 29.      , 404.      , 150.      , ...,  55.352417, 151.16054 ,
        100.700874],
       [ 59.      , 490.      , 150.      , ...,  55.352417, 151.16054 ,
        100.700874],
       ...,
       [ 30.95    , 595.      , 150.      , ...,  55.352417, 151.16054 ,
        100.700874],
       [117.      , 481.      , 150.      , ...,  55.352417, 151.16054 ,
        100.700874],
       [279.95    , 170.      , 150.      , ...,  55.352417, 151.16054 ,
        100.700874]], dtype=float32)

In [10]:
gc.collect()

1155